# Quantum circuit construction for commuting Pauli exponentials
## Introduction

The Pauli exponential is the operator defining as

\begin{equation}
e^{-i \alpha P / 2},
\tag{1}
\end{equation}

where the $N$-qubit Pauli string (PS) is given by

$$P = \sigma^{i_{N-1}}_{N-1}\otimes\dots\otimes\sigma^{i_0}_0$$

with $\sigma_i^j \in \{I,X,Y,Z\}$ standing for the Pauli matrix acting on $i$th qubit.
The procedure of transforming Pauli exponential into the qubit operations is well established and can be described as a tree-step process: 
1. diagonalize each $\sigma_i^j$ by single-qubit operations
2. store the required parity in one of the qubits and apply the single-qubit rotation $R_Z(\alpha)$ to it
3. restore the parity of the qubits

As an example, let us apply this procedure to the operator $\exp\{-i\frac{\alpha}{2} XIYZ\}$. For the sake of brevity, here and throughout we omit the symbol $\otimes$ from the expressions.
1. Utilizing the fact that 

\begin{equation}
X = H Z H,\qquad Y = V^\dagger Z V,
\end{equation}

with

\begin{equation}
H = \frac{1}{\sqrt{2}}\begin{pmatrix}1 & 1 \\ 1 & -1\end{pmatrix},
\qquad
V = R_X\left(\frac{\pi}{2}\right) = \frac{1}{\sqrt{2}}\begin{pmatrix}1 & -i \\ -i & 1\end{pmatrix},
\end{equation}

we write

\begin{equation}
e^{-i\frac{\alpha}{2} XIYZ} =
HIVI\ e^{-i\frac{\alpha}{2} ZIZZ}\ HIV^\dagger I
\end{equation}

2. To apply the operator $\exp\{-i\frac{\alpha}{2} ZIZZ\}$ we need first to collect the parity of 0,1, and 3 qubits into one of the qubits. This is done by the $CX$ gates acting on the pair of qubits as follows 

\begin{equation}
CX\left\vert x \right\rangle \left\vert y \right\rangle = 
\left\vert x \right\rangle \left\vert y\oplus x \right\rangle.
\end{equation}

In our case, the quantum circuit for collecting the required parity and storing it in the last qubit is

<img src="graph/example_parity.png" alt="drawing" width="300"/>

3. It is seen that on the previous step the parity of the qubits have changed. To restore it it is enough to apply the $CX$ gates in the reversed order. The overall circuit is then given by

<img src="graph/example_circuit.png" alt="drawing" width="350"/>

## The main part
In practice, however, operation (1) does not often appear. In most cases one needs to deal with

\begin{equation}
e^{-i\sum_i\alpha_i P_i}.
\tag{2}
\end{equation}

Here we concentrate on the situation when all $P_i$ commute with each other, i.e., $[P_i,P_j] = 0$. In this case, one can decompose operator (2) in the product of Pauli exponentials and transform each of them into the operations on qubits as is described above. But much more effective way is following
1. Diagonalize the whole set of commuting PS with a single operation as suggested in Ref. [1]. 
2. Collect the required parities and apply the single-qubit $R_Z$ gates in accordance to the algorithm suggested in Ref. [2].
3. Restore the parity by the algorithm from Ref. [3] which was proven to be optimal

Let us demonstrate this procedure on an example of the double-excitation operator, appearing in the decoupled Unitary Coupled Cluster approach for the quantum chemistry simulation. The double-excitation operator in the Jordan-Wigner mapping [4] is given by 

\begin{equation}
D =
e^{\alpha\left(a_3^\dagger a_2^\dagger a_2 a_0 - a_0^\dagger a_1^\dagger a_2 a_3 \right)} = e^{i\alpha\left(-YXXX-XYXX+XXYX-YYYX+XXXY-YYXY+YXYY+XYYY\right)/8},
\tag{3}
\end{equation}

where $a_p$ and $a_p^\dagger$ designate the annihilation and creation operators of the electron in the $p$ state, respectively. 
1. After the first step one obtains

\begin{equation}
D = U^\dagger e^{i\alpha\left(-ZIII +IZII -IZZI + ZIZI -IZZZ +ZIZZ -ZIIZ +IZIZ\right)/8} U,
\end{equation}

where $U$ is defined by the quatnum circuit

<img src="graph/U_double.png" alt="drawing" width="250"/>

2. Collection of all necessary parities and application of the $R_Z$ rotations in accordance to Ref. [2] is given by 

<img src="graph/parity_collect.png" alt="drawing" width="700"/>

Here the parities of the qubits are also indicated.

3. To restore the parity we apply the algorithm from Ref. [3]

<img src="graph/restore.png" alt="drawing" width="300"/>

The complete quantum circuit, which realises the double-excitation (3) is 

<img src="graph/double_complete.png" alt="drawing" width="900"/>

In this circuit there are 16 CNOT gates and 22 single-qubit ones. Note that the naive mapping wll give 40 CNOT and 56 single-qubit gates. 

For more details on launching the code look the [example](example.ipynb).

### Quadruple excitation
These algorithms were also launched to simplify the operator of the quadruple excitation. In the Jordan-Wigner mapping, this excitation consists of 128 PS acting on 8 qubits at least. The explicit form of PS with coefficients is given in [quadruple.inp](quadruple.inp). Naive translation to the qubit operations will give 2176 single-qubit and 1792 CNOT gates meanwhile the method described above will give the circuit with 158 single-qubit and 148 CNOT gates!

## Remarks
If the PS in (2) do not commute with each other one can proceed as follows. First, one needs to divide the set of the Pauli strings into several subsets with all the terms inside the subset commuting with each other. The next step is to apply one of the Suzuki-Trotter approximating formulas [5]. As a result, one will obtain the product of the operations of form (2) with commuting PS.

## References
[1] A. Cowtan, W. Simmons, and R. Duncan, [arXiv:2007.10515](https://arxiv.org/abs/2007.10515)

[2] V. Vandaele, S. Martiel, and T.G. de Brugiere, [arXiv:2104.00934](https://arxiv.org/abs/2104.00934)

[3] K.N. Patel, I.L. Markov, and J.P.Hayes, [Quantum Inf. and Comp. 8, 0282 (2008)](http://dx.doi.org/10.26421/QIC8.3-4-4).

[4] P. Jordan and E. Wigner, Z. Phys. A. 47, 631 (1928).

[5] M. Suzuki, Phys. Lett. A 146, 319 (1990).